# Level 10: CRUD Operations with SQLAlchemy ORM

This notebook covers the four fundamental database operations—Create, Read, Update, and Delete (CRUD)—using the SQLAlchemy ORM. All interactions with the database in the ORM are handled through a **Session** object.

### Setup
First, let's redefine our `User` model and set up the database engine, just as we did in the previous notebook. Each notebook should be runnable on its own.

In [1]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker
import os

db_file = 'sqlalchemy_crud.db'
if os.path.exists(db_file):
    os.remove(db_file)

engine = create_engine(f'sqlite:///{db_file}')
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)

    def __repr__(self):
        return f"<User(id={self.id}, name='{self.name}', age={self.age})>"

# Create the table
Base.metadata.create_all(engine)

## 10.1 Creating a Session

The **Session** is your primary interface for talking to the database. It's like a more powerful version of the `connection` and `cursor` objects from `sqlite3`.

You create a `Session` class using a `sessionmaker`, which is a factory for creating new Session objects.

In [2]:
# Create a Session factory and bind it to our engine
Session = sessionmaker(bind=engine)

# Create an instance of the Session
session = Session()
print("Session created.")

Session created.


## 10.2 Create (Insert)

To insert a new record, you create an instance of your model class and add it to the session.

In [3]:
# 1. Create a Python object
new_user = User(name='Alice', age=30)

# 2. Add it to the session (this stages the object for insertion)
session.add(new_user)

# 3. Commit the transaction to save it to the database
session.commit()

print(f"Created new user: {new_user}")

Created new user: <User(id=1, name='Alice', age=30)>


## 10.3 Read (Query)

You use the `session.query()` method to retrieve objects from the database.

In [4]:
# Add a few more users for our queries
session.add_all([
    User(name='Bob', age=35),
    User(name='Charlie', age=25)
])
session.commit()

In [5]:
# Get all users
all_users = session.query(User).all()
print("All users:", all_users)

All users: [<User(id=1, name='Alice', age=30)>, <User(id=2, name='Bob', age=35)>, <User(id=3, name='Charlie', age=25)>]


In [6]:
# Get the first user that matches a filter
alice = session.query(User).filter(User.name == 'Alice').first()
print("\nFound Alice:", alice)


Found Alice: <User(id=1, name='Alice', age=30)>


In [7]:
# Get users older than 28
older_users = session.query(User).filter(User.age > 28).all()
print("\nUsers older than 28:", older_users)


Users older than 28: [<User(id=1, name='Alice', age=30)>, <User(id=2, name='Bob', age=35)>]


## 10.4 Update

To update a record, you first query for it, then modify the attributes of the Python object, and finally commit the session.

In [8]:
# 1. Find the user to update
user_to_update = session.query(User).filter(User.name == 'Alice').first()
print(f"User before update: {user_to_update}")

# 2. Modify the object's attribute
user_to_update.age = 31

# 3. Commit the session. SQLAlchemy detects the change and issues an UPDATE statement.
session.commit()

print(f"User after update: {user_to_update}")

User before update: <User(id=1, name='Alice', age=30)>
User after update: <User(id=1, name='Alice', age=31)>


## 10.5 Delete

To delete a record, you query for it and then pass the object to `session.delete()`.

In [9]:
# 1. Find the user to delete
user_to_delete = session.query(User).filter(User.name == 'Charlie').first()
print(f"User to delete: {user_to_delete}")

# 2. Delete the object from the session
session.delete(user_to_delete)

# 3. Commit the change
session.commit()

# Verify deletion
charlie_exists = session.query(User).filter(User.name == 'Charlie').first()
print(f"Is Charlie still in the database? {charlie_exists}")

User to delete: <User(id=3, name='Charlie', age=25)>
Is Charlie still in the database? None


In [10]:
# Close the session when you're done
session.close()